In [46]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

torch.manual_seed(43)
device

'cuda'

In [47]:
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root="data",
    train=True,
    transform=ToTensor(),
    download=True,
)

test_data = datasets.MNIST(root="data", train=False, transform=ToTensor())
x_train_not_normalized = train_data.data.float().to(device=device)
y_train = train_data.targets.to(device=device)
x_test_not_normalized = test_data.data.float().to(device=device)
y_test = test_data.targets.to(device=device)

# normalization
x_train = (x_train_not_normalized - x_train_not_normalized.min()) / (
    x_train_not_normalized.max() - x_train_not_normalized.min()
)
x_test = (x_test_not_normalized - x_test_not_normalized.min()) / (
    x_test_not_normalized.max() - x_test_not_normalized.min()
)

train_loader = DataLoader(dataset=list(zip(x_train, y_train)), batch_size=32)
test_loader = DataLoader(dataset=list(zip(x_test, y_test)), batch_size=32)

In [48]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [49]:
import torch.optim as optim

student = Student().to("cuda")
optimizer = optim.SGD(student.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [50]:
for epoch in range(10):
    running_loss = 0.0

    for data in train_loader:
        inputs, truth = data
        optimizer.zero_grad()
        pred = student(inputs.to("cuda").float())
        truth = truth.to("cuda")
        loss = criterion(pred, truth)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss:{running_loss}")

Epoch 1, Loss:2188.2704999297857
Epoch 2, Loss:730.2676546275616
Epoch 3, Loss:597.0679777469486
Epoch 4, Loss:520.4293486177921
Epoch 5, Loss:461.14736443571746
Epoch 6, Loss:412.1882497901097
Epoch 7, Loss:371.9845690159127
Epoch 8, Loss:338.5877961451188
Epoch 9, Loss:309.98394336085767
Epoch 10, Loss:285.16330262832344


In [51]:
misclassifications = 0
student.eval()
with torch.no_grad():
    for data in test_loader:
        inputs, truth = data
        optimizer.zero_grad()
        pred = student(inputs.float())
        pred = torch.argmax(pred, dim=1).float()
        truth = truth.float()
        misclassifications += torch.sum(pred != truth).item()

In [52]:
misclassifications

456

In [53]:
len(test_loader) * 32

10016

In [57]:
# print(f'Accuracy: {student(test_loader)}')
# misclassifications / (len(test_loader) * 32)
test_logits = student(x_test)

misclassified = (pred != y_test).count_nonzero().item()
_, pred = torch.softmax(test_logits, dim=1).max(dim=1)
print(
    f"Corectly Classified :{y_test.__len__() - misclassifications}\nMisclassified: {misclassifications}"
)

Corectly Classified :9544
Misclassified: 456


# Almost 0.89% misclassification
